# From illusory randomness to quantum power: how quantum computing redefines randomness and revolutionises machine learning with QSVM

### Explore how quantum computing overcomes the limitations of classical random number generation, opening new frontiers in security, simulation and machine learning. From true randomness to the implementation of QSVM, discover how this new quantum paradigm is transforming machine learning and revolutionising artificial intelligence.

-------

Autor: Bruno Muñoz Marcos



*Imagine a world where everything follows a predictable sequence, where uncertainty is just an illusion and nothing is truly random. This is the realm of classical computing, the mainstay of our digital age, where computers, governed by deterministic processes, face intrinsic limitations in generating genuine random numbers. Yet randomness is an essential concept for disciplines such as cryptography, the simulation of complex systems and, increasingly, artificial intelligence.
How far can our classical computation go if it cannot even produce a truly unpredictable number? To transcend this limitation, we must enter a world where uncertainty is a fundamental feature: quantum mechanics. Here, particles exist in multiple states simultaneously, and measurement itself affects the system, introducing a source of true randomness. It is in this domain that quantum computing has the potential to radically change our capabilities, from random number generation to the revolution in machine learning and artificial intelligence.*

## 

In [27]:
# Qiskit: essential elements
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendSampler

# Qiskit IBM Runtime
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Qiskit Machine Learning
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

# Scikit-learn utilities
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# NumPy and Matplotlib
import numpy as np
import matplotlib.pyplot as plt


## Pseudo-random numbers generator

In [ ]:
def lcg(a, c, m, seed, n):
    """Linear Congruential Generator (LCG).

    Args:
        a (int): Multiplier.
        c (int): Increment.
        m (int): Modulus.
        seed (int): Initial seed.
        n (int): Number of numbers to generate.

    Returns:
        list: List of pseudo-random numbers in [0, 1).
    """
    x = seed
    random_numbers = []
    for _ in range(n):
        x = (a * x + c) % m
        random_numbers.append(x / m)
    return random_numbers

# LCG parameters
a = 1103515245
c = 12345
m = 2**31
seed = 42
n = 10

# Generating pseudo-random numbers
random_numbers = lcg(a, c, m, seed, n)
print("Generated pseudo-random numbers:")
for num in random_numbers:
    print(num)


## Quantum random numbers generator

### With simulator:

In [ ]:
def generate_n_qubit_random_numbers(n_qubits, n_numbers):
    """Generates random numbers of n bits using n qubits."""
    simulator = AerSimulator()  
    random_numbers = []

    for _ in range(n_numbers):
        qc = QuantumCircuit(n_qubits, n_qubits)
        qc.h(range(n_qubits)) 
        qc.measure(range(n_qubits), range(n_qubits)) 
        job = simulator.run(qc, shots=1, memory=True)
        result = job.result()
        measured_bits = result.get_memory()[0]
        random_number = int(measured_bits, 2)
        random_numbers.append(random_number)

    return random_numbers

# Generate 5 random numbers with 4 qubits
n_qubits = 6
n_numbers = 100000
random_numbers = generate_n_qubit_random_numbers(n_qubits, n_numbers)
print("Generated random numbers:")
print(random_numbers)


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(random_numbers, bins=10, color='skyblue', edgecolor='black')
plt.title("Histogram of random numbers generated by simulator")
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('foto.png')
plt.show()

### With real quantum computer:

In [3]:
# Initialising the IBM Quantum service
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='put here your own token'
)

n_qubits = 6
n_numbers = 100000

# Selecting a quantum backend with at least n qubits
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)

random_numbers = []
results =[] 

# Create a quantum circuit with n qubits and classical bits for measurement
qc = QuantumCircuit(n_qubits, n_qubits)

# Apply Hadamard gates to put qubits in superposition
qc.h(range(n_qubits))

# Measure all the qubits and store the results in the classical bits
qc.measure(range(n_qubits), range(n_qubits))

# Transpiling the circuit to be compatible with the selected backend
qc_transpiled = transpile(qc, backend=backend)

# Use the sampler within a session to execute the transpiled circuit
sampler = Sampler(mode=backend)  
job = sampler.run([qc_transpiled], shots=n_numbers)

# Obtain the results of the work
result = job.result()
    

In [ ]:
random_numbers = result[0].data.c.get_int_counts()

In [ ]:
numbers = list(random_numbers.keys())
counts = list(random_numbers.values())

plt.figure(figsize=(10, 6))
plt.hist(numbers, bins=10, weights=counts, color='skyblue', edgecolor='black')
plt.title("Histogram of random numbers generated by quantum computer")
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Machine Learning: Classical SVM

In [ ]:
# Generation of a synthetic dataset
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, random_state=42, n_clusters_per_class=1)

# Change labels from {0,1} to {-1,1}
y = np.where(y == 0, -1, 1)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Creation and training of the SVM model
model = svm.SVC(kernel='linear', C=1.0)
model.fit(X_train, y_train)

# Model evaluation
accuracy = model.score(X_test, y_test)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# Visualization of the data and the hyperplane
def plot_svm(model, X, y):
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', alpha=0.7)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # Create a grid to evaluate the model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model.decision_function(xy).reshape(XX.shape)

    # Draw the hyperplane and margins
    ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.7,
               linestyles=['--', '-', '--'])
    plt.savefig('SVM.png')
    plt.show()


plot_svm(model, X_train, y_train)


## Quantum Machine Learning: QSVM

In [ ]:
# 1. Generate a synthetic dataset for binary classification
num_samples = 50
rng = np.random.default_rng(seed=42)
X = rng.uniform(-1, 1, (num_samples, 2))
y = np.where(X[:, 0] + X[:, 1] > 0, 1, -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# 2. Define a quantum feature map
feature_dimension = 2
repetitions = 2
feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=repetitions)


# 3. Configure the backend and compute the kernel
backend = AerSimulator(method='statevector')
sampler = BackendSampler(backend=backend)
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)

# Compute the kernel matrices
kernel_train = quantum_kernel.evaluate(X_train)
kernel_test = quantum_kernel.evaluate(X_test, X_train)

# 4. Train an SVM classifier with a precomputed kernel
svc = SVC(kernel='precomputed')
svc.fit(kernel_train, y_train)


# 5. Evaluate the model on the test set
y_pred = svc.predict(kernel_test)
accuracy = accuracy_score(y_test, y_pred)

print("SVC predictions on the test set:", y_pred)
print("True labels:", y_test)
print("Accuracy:", accuracy)